In [1]:
import pandas as pd
import os
import sqlite3
import re
import html
import textacy.preprocessing as tprep
from textacy.preprocessing.resources import RE_URL
import nltk
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex

2023-01-08 20:37:43.141703: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 20:37:43.141736: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-08 20:37:45.024774: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-08 20:37:45.025490: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-08 20:37:45.026449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-08 20:37:45.026918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties

## 1. Data

In [2]:
os.chdir("../blueprints-text/")

In [3]:
data_path = "data"

In [4]:
from packages.blueprints.exploration import count_words

### 1.1 Get the data

In [5]:
posts_df = pd.read_csv(os.path.join(data_path, "reddit-selfposts/rspct_autos.tsv.gz"), sep="\t")

In [6]:
posts_df

,id,subreddit,title,selftext
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and ..."
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information...
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should...
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering bu..."
...,...,...,...,...
19995,7i2k6y,4Runner,Bilstein Shocks,I read a lot Forums and people recommend getti...
19996,83p2kv,Harley,Question on potential purchase of crashed bike.,I am thinking about buying a 2010 Harley Spor...
19997,7x722h,volt,Got our first warning light on our dash,My husband and I were headed somewhere and I w...
19998,7v2xmg,Lexus,Any IS models to avoid?,I am looking at getting a used Lexus IS (2014 ...


In [7]:
subred_df = pd.read_csv(os.path.join(data_path, "reddit-selfposts/subreddit_info.csv.gz")).set_index(["subreddit"])

In [8]:
subred_df

,category_1,category_2,category_3,in_data,reason_for_exclusion
subreddit,,,,,
whatsthatbook,advice/question,book,NaN,True,NaN
CasualConversation,advice/question,broad,NaN,False,too_broad
Clairvoyantreadings,advice/question,broad,NaN,False,too_broad
DecidingToBeBetter,advice/question,broad,NaN,False,too_broad
HelpMeFind,advice/question,broad,NaN,False,too_broad
...,...,...,...,...,...
HFY,writing/stories,sci-fi,NaN,True,NaN
TalesFromYourServer,writing/stories,tech support,NaN,False,fewer posts than r/talesfromtechsupport which ...
talesfromtechsupport,writing/stories,tech support,NaN,True,NaN


In [9]:
df = pd.merge(posts_df, subred_df, on="subreddit")

In [10]:
df

,id,subreddit,title,selftext,category_1,category_2,category_3,in_data,reason_for_exclusion
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson,NaN,True,NaN
1,4z6vee,Harley,Mystery Rattle,I have a stock 2010 FXD that has a rate I can'...,autos,harley davidson,NaN,True,NaN
2,6x4tdk,Harley,Gum on tailpipe,So I'm on a bike trip with my father and we st...,autos,harley davidson,NaN,True,NaN
3,7l7rer,Harley,Changing up the bars!,So I’m gonna change out my T bars that came on...,autos,harley davidson,NaN,True,NaN
4,683pc7,Harley,Spent 2 days and made some Mini Floorboards fo...,"A few weeks ago, somebody was looking for some...",autos,harley davidson,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...
19995,7346dx,4Runner,03 JBL Sound System Issue,Just picked up my first 4Runner this week! Lov...,autos,toyota,NaN,True,NaN
19996,5ut9e2,4Runner,2nd gen Comes Back From the Dead With No Tools,Hello fellow runners-<lb><lb>I wanted to share...,autos,toyota,NaN,True,NaN
19997,66xnwd,4Runner,Tranny cooler in cold temperatures?,So I picked up a 2002 4Runner recently and the...,autos,toyota,NaN,True,NaN
19998,8kvxkg,4Runner,Torn between a Wrangler and a 4 Runner...,Ok so I've posted to r/Jeep and figured I shou...,autos,toyota,NaN,True,NaN


In [11]:
column_mapping = {
    'id': 'id',
    'subreddit': 'subreddit',
    'title': 'title',
    'selftext': 'text',
    'category_1': 'category',
    'category_2': 'subcategory',
    'category_3': None,
    'in_data': None,
    'reason_for_exclusion': None
}

In [12]:
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]
columns

['id', 'subreddit', 'title', 'selftext', 'category_1', 'category_2']

In [13]:
df = df[columns].rename(columns=column_mapping)
df

,id,subreddit,title,text,category,subcategory
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson
1,4z6vee,Harley,Mystery Rattle,I have a stock 2010 FXD that has a rate I can'...,autos,harley davidson
2,6x4tdk,Harley,Gum on tailpipe,So I'm on a bike trip with my father and we st...,autos,harley davidson
3,7l7rer,Harley,Changing up the bars!,So I’m gonna change out my T bars that came on...,autos,harley davidson
4,683pc7,Harley,Spent 2 days and made some Mini Floorboards fo...,"A few weeks ago, somebody was looking for some...",autos,harley davidson
...,...,...,...,...,...,...
19995,7346dx,4Runner,03 JBL Sound System Issue,Just picked up my first 4Runner this week! Lov...,autos,toyota
19996,5ut9e2,4Runner,2nd gen Comes Back From the Dead With No Tools,Hello fellow runners-<lb><lb>I wanted to share...,autos,toyota
19997,66xnwd,4Runner,Tranny cooler in cold temperatures?,So I picked up a 2002 4Runner recently and the...,autos,toyota
19998,8kvxkg,4Runner,Torn between a Wrangler and a 4 Runner...,Ok so I've posted to r/Jeep and figured I shou...,autos,toyota


In [14]:
df = df[df.category == "autos"]
df

,id,subreddit,title,text,category,subcategory
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson
1,4z6vee,Harley,Mystery Rattle,I have a stock 2010 FXD that has a rate I can'...,autos,harley davidson
2,6x4tdk,Harley,Gum on tailpipe,So I'm on a bike trip with my father and we st...,autos,harley davidson
3,7l7rer,Harley,Changing up the bars!,So I’m gonna change out my T bars that came on...,autos,harley davidson
4,683pc7,Harley,Spent 2 days and made some Mini Floorboards fo...,"A few weeks ago, somebody was looking for some...",autos,harley davidson
...,...,...,...,...,...,...
19995,7346dx,4Runner,03 JBL Sound System Issue,Just picked up my first 4Runner this week! Lov...,autos,toyota
19996,5ut9e2,4Runner,2nd gen Comes Back From the Dead With No Tools,Hello fellow runners-<lb><lb>I wanted to share...,autos,toyota
19997,66xnwd,4Runner,Tranny cooler in cold temperatures?,So I picked up a 2002 4Runner recently and the...,autos,toyota
19998,8kvxkg,4Runner,Torn between a Wrangler and a 4 Runner...,Ok so I've posted to r/Jeep and figured I shou...,autos,toyota


### 1.2 Save the data

In [15]:
df.to_pickle(os.path.join(data_path, "reddit-selfposts/reddit_dataframe.pkl"))

In [16]:
db_name = "reddit-selfposts.db"
con = sqlite3.connect(os.path.join(data_path, "reddit-selfposts", db_name))
df.to_sql("posts", con, index=False, if_exists="replace")
con.close()

In [17]:
con = sqlite3.connect(os.path.join(data_path, "reddit-selfposts", db_name))
pd.read_sql("SELECT * FROM posts", con)
con.close()

## 2. Cleaning text data

In [18]:
df.text

0        Funny story. I went to college in Las Vegas. T...
1        I have a stock 2010 FXD that has a rate I can'...
2        So I'm on a bike trip with my father and we st...
3        So I’m gonna change out my T bars that came on...
4        A few weeks ago, somebody was looking for some...
                               ...                        
19995    Just picked up my first 4Runner this week! Lov...
19996    Hello fellow runners-<lb><lb>I wanted to share...
19997    So I picked up a 2002 4Runner recently and the...
19998    Ok so I've posted to r/Jeep and figured I shou...
19999    I read a lot Forums and people recommend getti...
Name: text, Length: 20000, dtype: object

### 2.1 Blueprint: Identify noise with regular expressions

In [19]:
RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

In [20]:
def impurity(text, min_len=10):
    """
    returns the sahre of suspicious characters in a text
    """
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)

In [21]:
df['impurity'] = df['text'].apply(impurity, min_len=10)

In [22]:
df[["text", "impurity"]].sort_values(by="impurity", ascending=False).head(3)

,text,impurity
14983,Looking at buying a 335i with 39k miles and 11...,0.214716
8630,I'm looking to lease an a4 premium plus automa...,0.165099
15141,Breakdown below:<lb><lb>Elantra GT<lb><lb>2.0L...,0.139130


In [23]:
count_words(df, column='text', preprocess=lambda t: re.findall(r'<[\w/]*>', t))

,freq
token,
<lb>,100729
<tab>,642


### 2.2 Blueprint: Removing noise with regular expressions

In [24]:
def clean(text):
    # convert html escapes like &amp; to characters
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r"<[^<>]*>", " ", text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r"\[([^\[\]]*)\]\([^\(\)]*\)", r"\1", text)
    # text or code in brackets like [0]
    text = re.sub(r"\[[^\[\]]*\]", " ", text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r"(?:^|\s)[\-=\+]{2,}(?:\s|$)", " ", text)
    # sequences of white spaces
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [25]:
df["clean_text"] = df.text.apply(clean)

In [26]:
df["impurity"] = df.clean_text.apply(impurity, min_len=20)

In [27]:
df[["clean_text", "impurity"]].sort_values(by="impurity", ascending=False)

,clean_text,impurity
15361,Split b/w 2 genesis options. Hyundai Genesis\ ...,0.039867
1714,"Mustang 2018, 2019, or 2020? Must Haves!! 1. H...",0.031980
17941,"Just like the title says, when I am manually s...",0.028914
16959,"At the dealership, they offered an option for ...",0.026455
12828,"I am looking at four Caymans, all are in a sim...",0.024631
...,...,...
6866,Im lookin to get a 2008 honda fit with 150000k...,0.000000
6865,My car when I'm driving and know I'm going to ...,0.000000
6864,"Hey /r/Honda, Ever since upgrading to iOS 11 a...",0.000000
6863,I just bought a 97 civic with 105k miles on it...,0.000000


### 2.3 Blueprint: Character normalization with textacy

In [28]:
text = "The café “Saint-Raphaël” is loca-\nted on Côte dʼAzur."

In [29]:
def normalize(text):
    text = tprep.normalize.hyphenated_words(text)
    text = tprep.normalize.quotation_marks(text)
    text = tprep.normalize.unicode(text)
    text = tprep.remove.accents(text)
    return text

In [30]:
print(text)

The café “Saint-Raphaël” is loca-
ted on Côte dʼAzur.


In [31]:
print(normalize(text))

The cafe "Saint-Raphael" is located on Cote d'Azur.


### 2.4 Blueprint: Pattern-based data masking with textacy

In [32]:
count_words(df, column="clean_text")

,freq
token,
,2448392
e,1137060
t,925384
a,777509
o,758592
...,...
͡,2
à,2
😔,2


In [33]:
count_words(df, column="clean_text", preprocess=RE_URL.findall)

,freq
token,
www.getlowered.com,3
http://gm-volt.com,2
https://imgur.com/a/lnFec,2
https://imgur.com/gallery/rkdv4,2
https://www.reddit.com/r/Jeep/comments/4ux232/just_ordered_an_android_head_unit_joying_jeep/,2
http://imgur.com/3hd0dy9,2
http://ir.tesla.com,2
https://imgur.com/a/mbS08,2
http://www.ecolamautomotive.com/#!2/kv7fq,2


In [34]:
text = "Check out https://spacy.io/usage/spacy-101"
print(tprep.replace.urls(text))

Check out _URL_


In [35]:
df["clean_text"] = df.clean_text.map(tprep.replace.urls)
df["clean_text"] = df.clean_text.map(normalize)

In [36]:
df.clean_text

0        Funny story. I went to college in Las Vegas. T...
1        I have a stock 2010 FXD that has a rate I can'...
2        So I'm on a bike trip with my father and we st...
3        So I'm gonna change out my T bars that came on...
4        A few weeks ago, somebody was looking for some...
                               ...                        
19995    Just picked up my first 4Runner this week! Lov...
19996    Hello fellow runners- I wanted to share my rec...
19997    So I picked up a 2002 4Runner recently and the...
19998    Ok so I've posted to r/Jeep and figured I shou...
19999    I read a lot Forums and people recommend getti...
Name: clean_text, Length: 20000, dtype: object

In [37]:
df.rename(columns={"text": "raw_text", "clean_text": "text"}, inplace=True)
df.drop(columns=["impurity"], inplace=True)

In [38]:
con = sqlite3.connect(os.path.join(data_path, "reddit-selfposts", db_name))
df.to_sql("posts_cleaned", con, index=False, if_exists="replace")
con.close()

In [39]:
con = sqlite3.connect(os.path.join(data_path, "reddit-selfposts", db_name))
pd.read_sql("SELECT * FROM posts_cleaned", con)
con.close()

## 3. Tokenization

In [40]:
text = """
2019-08-10 23:32: @pete/@louis - I don't have a well-designed 
solution for today's problem. The code of module AC68 should be -1. 
Have to think a bit... #goodnight ;-) 😩😬"""

### 3.1 Blueprint: Tokenization with regular expressions

In [41]:
tokens = re.findall(r"\w\w+", text)
print(*tokens, sep="|")

2019|08|10|23|32|pete|louis|don|have|well|designed|solution|for|today|problem|The|code|of|module|AC68|should|be|Have|to|think|bit|goodnight


In [42]:
RE_TOKEN = re.compile(r"""
               ( [#]?[@\w'’\.\-\:]*\w     # words, hash tags and email adresses
               | [:;<]\-?[\)\(3]          # coarse pattern for basic text emojis
               | [\U0001F100-\U0001FFFF]  # coarse code range for unicode emojis
               )
               """, re.VERBOSE)

def tokenize(text):
    return RE_TOKEN.findall(text)

tokens = tokenize(text)
print(*tokens, sep='|')

2019-08-10|23:32|@pete|@louis|I|don't|have|a|well-designed|solution|for|today's|problem|The|code|of|module|AC68|should|be|-1|Have|to|think|a|bit|#goodnight|;-)|😩|😬


### 3.2 Tokenization with NLTK

In [43]:
tokens = nltk.tokenize.word_tokenize(text)
print(*tokens, sep="|")

2019-08-10|23:32|:|@|pete/|@|louis|-|I|do|n't|have|a|well-designed|solution|for|today|'s|problem|.|The|code|of|module|AC68|should|be|-1|.|Have|to|think|a|bit|...|#|goodnight|;|-|)|😩😬


In [44]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nicolas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
# Not in book: Tweet Tokenizer
tokenizer = nltk.tokenize.TweetTokenizer()
tokens = tokenizer.tokenize(text)
print(*tokens, sep='|')

2019-08-|10|23:32|:|@pete|/|@louis|-|I|don't|have|a|well-designed|solution|for|today's|problem|.|The|code|of|module|AC68|should|be|-|1|.|Have|to|think|a|bit|...|#goodnight|;-)|😩|😬


## 4. Linguistic processing with spacy

In [46]:
!python -m spacy download en_core_web_sm

2023-01-08 20:37:58.694715: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 20:37:58.694740: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-08 20:37:59.810297: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-08 20:37:59.810345: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-08 20:37:59.810453: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-08 20:37:59.810861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [47]:
nlp = spacy.load("en_core_web_sm")

In [48]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f14db8bcd60>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7f14db89d820>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f14dc41ca50>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f15d9249fc0>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7f15d9249b40>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f14dc41c9e0>)]

In [49]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f15cde25580>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7f15cdca6be0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f15cdc27180>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7f15cdc2f080>)]

### 4.1 Processing Text

In [50]:
text = "My best friend Ryan Peters likes fancy adventure games."

In [51]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

In [52]:
doc

My best friend Ryan Peters likes fancy adventure games.

In [53]:
dir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_context',
 '_get_array_attrs',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'from_json',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set

In [54]:
for token in doc:
    print(token, end="|")

My|best|friend|Ryan|Peters|likes|fancy|adventure|games|.|

In [55]:
doc

My best friend Ryan Peters likes fancy adventure games.

In [56]:
doc.text

'My best friend Ryan Peters likes fancy adventure games.'

In [57]:
for t in doc:
    print(t.is_stop, t.pos_)

True PRON
False ADJ
False NOUN
False PROPN
False PROPN
False VERB
False ADJ
False NOUN
False NOUN
False PUNCT


### 4.2 Customizing tokenization

In [58]:
text = "@Pete: choose low-carb #food #eat-smart. _url_ ;-) 😋👍"

In [59]:
doc = nlp(text)

In [60]:
for token in doc:
    print(token, end="|")

@Pete|:|choose|low|-|carb|#|food|#|eat|-|smart|.|_|url|_|;-)|😋|👍|

In [61]:
for x in nlp.vocab:
    print(x.text)

nuthin
ü.
p.m
Kan
Mar
When's
 
Sept.
c.
Mont.
:-}
12a.m.
e.g
Why's
😋
it
6p.m
Jr.
Who’s
K.
Calif.
e
Ill.
O'clock
o'clock
Mich.
is
:-o
n.
Might
Nov
>.<
he's
it’s
where’s
low
Wash.
where
:-*
she's
g.
:()
)-:
X
S.C.
Del
Why’s
0.o
must
Goin'
4a.m.
5p.m.
Mass.
co.
━
(-_-)
Ariz
had
0
vs.
x.
><(((*>
11a.m.
-o
When’s
Calif
does
nothin’
’S
Cos
I.e
8-)
Would
do
"
Rev.
’s
N.M.
°c.
b
O.o
might
q.
adventure
this's
’’
Goin’
likes
Has
N.J.
pm
ought
Dec
food
3p.m
<space>
Ore.
10p.m
h.
where's
doin’
What's
he’s
'cos
ä
That's
11a.m
these
1p.m
Tenn
;D
Miss
Ga
Must
9p.m
somethin
Nev.
What
z.
’
there's
'Cause
ü
r
9
Dec.
︵
v.s
Jun.
:-))
;_;
When
D.C.
Have
You
10
Where's
6a.m
o_o
O
'coz
'cause
co
doin
how's
i.e
._.
c’m
Prof.
:’-)
havin'
Co.
Va.
Mt
|
:((
O’clock
Let
Del.
xDD
Prof
i.e.
Ai
lovin'
when’s
Does
Minn
g
👍
12a.m
Who's
-
(
La
11
(;
12p.m.
Mich
n’t
_
Not
choose
1p.m.
Okla.
Okla
what's
□
:P
w
cos
:'-(
Ph.D.
It’s
Va
Nothin’
Ltd.
would
:-/
Rep
Was
;
a.
:)))
k
o.
XD
#
Ga.
Sha
C
Ought
Sep
>:o
:-)
<3
d
8)
Who

In [62]:
def custom_tokenizer(nlp):
    # use default patterns except the ones mathced by re.search
    prefixes = [pattern for pattern in nlp.Defaults.prefixes if pattern not in ["-", "_", "#"]]
    suffixes = [pattern for pattern in nlp.Defaults.suffixes if pattern not in ["_"]]
    infixes = [pattern for pattern in nlp.Defaults.infixes if not re.search(pattern, "xx-xx")]
    return Tokenizer(
        vocab = nlp.vocab,
        rules = nlp.Defaults.tokenizer_exceptions,
        prefix_search = compile_prefix_regex(prefixes).search,
        suffix_search = compile_suffix_regex(suffixes).search,
        infix_finditer = compile_infix_regex(infixes).finditer,
        token_match = nlp.Defaults.token_match
    )

In [63]:
nlp.Defaults.prefixes

['§',
 '%',
 '=',
 '—',
 '–',
 '\\+(?![0-9])',
 '…',
 '……',
 ',',
 ':',
 ';',
 '\\!',
 '\\?',
 '¿',
 '؟',
 '¡',
 '\\(',
 '\\)',
 '\\[',
 '\\]',
 '\\{',
 '\\}',
 '<',
 '>',
 '_',
 '#',
 '\\*',
 '&',
 '。',
 '？',
 '！',
 '，',
 '、',
 '；',
 '：',
 '～',
 '·',
 '।',
 '،',
 '۔',
 '؛',
 '٪',
 '\\.\\.+',
 '…',
 "\\'",
 '"',
 '”',
 '“',
 '`',
 '‘',
 '´',
 '’',
 '‚',
 ',',
 '„',
 '»',
 '«',
 '「',
 '」',
 '『',
 '』',
 '（',
 '）',
 '〔',
 '〕',
 '【',
 '】',
 '《',
 '》',
 '〈',
 '〉',
 '\\$',
 '£',
 '€',
 '¥',
 '฿',
 'US\\$',
 'C\\$',
 'A\\$',
 '₽',
 '﷼',
 '₴',
 '₠',
 '₡',
 '₢',
 '₣',
 '₤',
 '₥',
 '₦',
 '₧',
 '₨',
 '₩',
 '₪',
 '₫',
 '€',
 '₭',
 '₮',
 '₯',
 '₰',
 '₱',
 '₲',
 '₳',
 '₴',
 '₵',
 '₶',
 '₷',
 '₸',
 '₹',
 '₺',
 '₻',
 '₼',
 '₽',
 '₾',
 '₿',
 '[\\u00A6\\u00A9\\u00AE\\u00B0\\u0482\\u058D\\u058E\\u060E\\u060F\\u06DE\\u06E9\\u06FD\\u06FE\\u07F6\\u09FA\\u0B70\\u0BF3-\\u0BF8\\u0BFA\\u0C7F\\u0D4F\\u0D79\\u0F01-\\u0F03\\u0F13\\u0F15-\\u0F17\\u0F1A-\\u0F1F\\u0F34\\u0F36\\u0F38\\u0FBE-\\u0FC5\\u0FC7-\\u0FCC\\u0F

In [64]:
nlp.Defaults.infixes

['\\.\\.+',
 '…',
 '[\\u00A6\\u00A9\\u00AE\\u00B0\\u0482\\u058D\\u058E\\u060E\\u060F\\u06DE\\u06E9\\u06FD\\u06FE\\u07F6\\u09FA\\u0B70\\u0BF3-\\u0BF8\\u0BFA\\u0C7F\\u0D4F\\u0D79\\u0F01-\\u0F03\\u0F13\\u0F15-\\u0F17\\u0F1A-\\u0F1F\\u0F34\\u0F36\\u0F38\\u0FBE-\\u0FC5\\u0FC7-\\u0FCC\\u0FCE\\u0FCF\\u0FD5-\\u0FD8\\u109E\\u109F\\u1390-\\u1399\\u1940\\u19DE-\\u19FF\\u1B61-\\u1B6A\\u1B74-\\u1B7C\\u2100\\u2101\\u2103-\\u2106\\u2108\\u2109\\u2114\\u2116\\u2117\\u211E-\\u2123\\u2125\\u2127\\u2129\\u212E\\u213A\\u213B\\u214A\\u214C\\u214D\\u214F\\u218A\\u218B\\u2195-\\u2199\\u219C-\\u219F\\u21A1\\u21A2\\u21A4\\u21A5\\u21A7-\\u21AD\\u21AF-\\u21CD\\u21D0\\u21D1\\u21D3\\u21D5-\\u21F3\\u2300-\\u2307\\u230C-\\u231F\\u2322-\\u2328\\u232B-\\u237B\\u237D-\\u239A\\u23B4-\\u23DB\\u23E2-\\u2426\\u2440-\\u244A\\u249C-\\u24E9\\u2500-\\u25B6\\u25B8-\\u25C0\\u25C2-\\u25F7\\u2600-\\u266E\\u2670-\\u2767\\u2794-\\u27BF\\u2800-\\u28FF\\u2B00-\\u2B2F\\u2B45\\u2B46\\u2B4D-\\u2B73\\u2B76-\\u2B95\\u2B98-\\u2BC8\\u2BCA-\\

In [65]:
len(nlp.Defaults.infixes), len([pattern for pattern in nlp.Defaults.infixes if not re.search(pattern, "xx-xx")])

(8, 7)

In [66]:
[pattern for pattern in nlp.Defaults.infixes if re.search(pattern, "xx-xx")]

['(?<=[A-Za-z\\uFF21-\\uFF3A\\uFF41-\\uFF5A\\u00C0-\\u00D6\\u00D8-\\u00F6\\u00F8-\\u00FF\\u0100-\\u017F\\u0180-\\u01BF\\u01C4-\\u024F\\u2C60-\\u2C7B\\u2C7E\\u2C7F\\uA722-\\uA76F\\uA771-\\uA787\\uA78B-\\uA78E\\uA790-\\uA7B9\\uA7FA\\uAB30-\\uAB5A\\uAB60-\\uAB64\\u0250-\\u02AF\\u1D00-\\u1D25\\u1D6B-\\u1D77\\u1D79-\\u1D9A\\u1E00-\\u1EFFёа-яЁА-ЯәөүҗңһӘӨҮҖҢҺα-ωάέίόώήύΑ-ΩΆΈΊΌΏΉΎа-щюяіїєґА-ЩЮЯІЇЄҐѓѕјљњќѐѝЃЅЈЉЊЌЀЍ\\u1200-\\u137F\\u0980-\\u09FF\\u0591-\\u05F4\\uFB1D-\\uFB4F\\u0620-\\u064A\\u066E-\\u06D5\\u06E5-\\u06FF\\u0750-\\u077F\\u08A0-\\u08BD\\uFB50-\\uFBB1\\uFBD3-\\uFD3D\\uFD50-\\uFDC7\\uFDF0-\\uFDFB\\uFE70-\\uFEFC\\U0001EE00-\\U0001EEBB\\u0D80-\\u0DFF\\u0900-\\u097F\\u0C80-\\u0CFF\\u0B80-\\u0BFF\\u0C00-\\u0C7F\\uAC00-\\uD7AF\\u1100-\\u11FF\\u3040-\\u309F\\u30A0-\\u30FFー\\u4E00-\\u62FF\\u6300-\\u77FF\\u7800-\\u8CFF\\u8D00-\\u9FFF\\u3400-\\u4DBF\\U00020000-\\U000215FF\\U00021600-\\U000230FF\\U00023100-\\U000245FF\\U00024600-\\U000260FF\\U00026100-\\U000275FF\\U00027600-\\U000290FF\\U0002910

In [67]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for token in doc:
    print(token, end="|")

@Pete|:|choose|low|-|carb|#|food|#|eat|-|smart|.|_|url|_|;-)|😋|👍|

In [68]:
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = custom_tokenizer(nlp)

In [69]:
doc = nlp(text)
for token in doc:
    print(token, end="|")

@Pete|:|choose|low-carb|#food|#eat-smart|.|_url_|;-)|😋|👍|

### 4.3 Blueprint: Working with stop words

In [70]:
text = "Dear Ryan, we need to sit down and talk. Regards, Pete"

In [71]:
doc = nlp(text)

In [72]:
non_stop = [t for t in doc if not t.is_stop and not t.is_punct]

In [73]:
non_stop

[Dear, Ryan, need, sit, talk, Regards, Pete]

In [74]:
nlp = spacy.load("en_core_web_sm")

In [75]:
nlp.vocab["down"].is_stop = False
nlp.vocab["Dear"].is_stop = True
nlp.vocab["Regards"].is_stop = True

In [76]:
doc = nlp(text)

In [77]:
[t for t in doc if not t.is_stop and not t.is_punct]

[Ryan, need, sit, down, talk, Pete]

### 4.4 Blueprint: Extracting lemmas based on part-of-speech

In [182]:
text = "My best friend Ryan Peters likes fancy adventure games."

In [183]:
doc = nlp(text)
doc

My best friend Ryan Peters likes fancy adventure games.

In [84]:
print(*[t.lemma_ for t in doc], sep="|")

my|good|friend|Ryan|Peters|like|fancy|adventure|game|.


In [81]:
for i in doc:
    print(i, i.pos_, i.tag_)

My PRON PRP$
best ADJ JJS
friend NOUN NN
Ryan PROPN NNP
Peters PROPN NNP
likes VERB VBZ
fancy ADJ JJ
adventure NOUN NN
games NOUN NNS
. PUNCT .


In [82]:
nouns = [t for t in doc if t.pos_ in ["NOUN", "PROPN"]]
nouns

[friend, Ryan, Peters, adventure, games]

In [94]:
import textacy

In [110]:
tokens = textacy.extract.words(
    doc, 
    filter_stops=True, # stopwords
    filter_punct=True, # punctuation
    filter_nums=True, # numbers
    include_pos=['Adj', 'NOUN'], # Include Part-of-Speech
    exclude_pos=None, # Exclude Part-of-Speech
    min_freq=1 # minimum frequency of words
)

In [111]:
print(*[t for t in tokens], sep="|")

best|friend|fancy|adventure|games


In [184]:
def extract_lemmas(doc, **kwargs):
    return [t .lemma_ for t in textacy.extract.words(doc, **kwargs)]

In [187]:
lemmas = extract_lemmas(doc, include_pos=["Adj", "NOUN"])

In [188]:
lemmas 

['good', 'friend', 'fancy', 'adventure', 'game']

In [189]:
print(*lemmas, sep="|")

good|friend|fancy|adventure|game


### 4.5 Blueprint: Extracting Noun Phrases

In [112]:
text

'My best friend Ryan Peters likes fancy adventure games.'

In [113]:
doc = nlp(text)

In [114]:
doc

My best friend Ryan Peters likes fancy adventure games.

In [141]:
patterns = ["POS:ADJ POS:NOUN:+"]
spans = textacy.extract.token_matches(doc, patterns=patterns)

In [142]:
textacy.extract.matches.token_matches(doc, patterns=patterns)

<generator object token_matches at 0x7f14d2fd4190>

In [143]:
print(*[s.lemma_ for s in spans], sep="|")

good friend|fancy adventure|fancy adventure game


In [159]:
def extract_noun_phrases(doc, preceeding_pos=["Noun"], sep="_"):
    patterns=[]
    for pos in preceeding_pos:
        patterns.append(f"POS:{pos} POS:NOUN:+")
    spans = textacy.extract.token_matches(doc, patterns=patterns)
    return [sep.join([t.lemma_ for t in s]) for s in spans]

In [160]:
print(*extract_noun_phrases(doc, preceeding_pos=["NOUN", "ADJ"]), sep="|")

good_friend|fancy_adventure|fancy_adventure_game|adventure_game


### 4.6 Blueprint: Extracting Named Entities

In [161]:
text = "James O'Neill, chairman of World Cargo Inc, lives in San Francisco."
doc = nlp(text)

In [163]:
for ent in doc.ents:
    print(f"({ent.text}, {ent.label_})")

(James O'Neill, PERSON)
(World Cargo Inc, ORG)
(San Francisco, GPE)


In [164]:
from spacy import displacy

In [165]:
displacy.render(doc, style="ent")

In [178]:
def extract_entities(doc, include_types=None, sep="_"):
    ents = textacy.extract.entities(
        doc,
        include_types=include_types,
        exclude_types=None,
        drop_determiners=True,
        min_freq=1
    )
    return [sep.join([t.lemma_ for t in e])+"/"+e.label_ for e in ents]

In [179]:
extract_entities(doc)

["James_O'Neill/PERSON", 'World_Cargo_Inc/ORG', 'San_Francisco/GPE']

## 5. Feature Extraction on a Large Dataset

### 5.1 Blueprint: Combine Extraction Function into One Function to Get it All

In [197]:
def extract_nlp(doc):
    return {
        'lemmas': extract_lemmas(doc, exclude_pos=["PART", "PUNCT", "DET", "PRON", "SYM", "SPACE"], filter_stops=False),
        'adj_verbs': extract_lemmas(doc, include_pos=["ADJ", "VERB"]),
        'nouns': extract_lemmas(doc, include_pos=["NOUN", "PROPN"]),
        'noun_phrases': extract_noun_phrases(doc, preceeding_pos=["NOUN"]),
        'adj_noun_phrases': extract_noun_phrases(doc, preceeding_pos=["ADJ"]),
        'entities': extract_entities(doc, ["PERSON", "ORG", "GPE", "LOC"])
    }
    

In [199]:
extract_nlp(doc)

{'lemmas': ['good',
  'friend',
  'Ryan',
  'Peters',
  'like',
  'fancy',
  'adventure',
  'game'],
 'adj_verbs': ['good', 'like', 'fancy'],
 'nouns': ['friend', 'Ryan', 'Peters', 'adventure', 'game'],
 'noun_phrases': ['adventure_game'],
 'adj_noun_phrases': ['good_friend',
  'fancy_adventure',
  'fancy_adventure_game'],
 'entities': ['Ryan_Peters/PERSON']}

In [200]:
for col, values in extract_nlp(doc).items():
    print(f"{col}: {values}")

lemmas: ['good', 'friend', 'Ryan', 'Peters', 'like', 'fancy', 'adventure', 'game']
adj_verbs: ['good', 'like', 'fancy']
nouns: ['friend', 'Ryan', 'Peters', 'adventure', 'game']
noun_phrases: ['adventure_game']
adj_noun_phrases: ['good_friend', 'fancy_adventure', 'fancy_adventure_game']
entities: ['Ryan_Peters/PERSON']


In [201]:
nlp_columns = list(extract_nlp(nlp.make_doc('')).keys())

In [202]:
nlp_columns

['lemmas',
 'adj_verbs',
 'nouns',
 'noun_phrases',
 'adj_noun_phrases',
 'entities']

### 5. 2 Blueprint: Usinf spaCy on a Large Dataset